<h1 align="center">Lab 2:  Sexism Identification in Twitter</h1>
<h2 align="center">Session 2. Transformers: Fine-tuning for multi-class classification
<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

### Put your names here

- Mateusz Król
- (replace with another name)

In [ ]:
!pip install transformers --upgrade
!pip install datasets accelerate --upgrade
!pip install peft --upgrade
!pip install jupyter --upgrade
!pip install ipywidgets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 91.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

## Many libraries

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoTokenizer, AutoModelForSequenceClassification,  Trainer, TrainingArguments,  EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random
import os
import pandas as pd
import sys
import tempfile
import time

In [ ]:
# IF YOU USE GOOGLE COLAB -> COLAB=True
COLAB = True

In [ ]:
if COLAB is True:
  from google.colab import drive
  drive.mount('/content/drive')
  base_path = "/content/drive/MyDrive/LNR/"
else:
  base_path = ".."
# print(base_path)

Mounted at /content/drive


## Import readerEXIST2025 library

In [ ]:
library_path = os.path.join(base_path, "Lab2-S1")
sys.path.append(library_path)
from readerEXIST2025 import EXISTReader

## Read dataset

In [ ]:
# path to the dataset, adapt this path wherever you have the dataset
# dataset_path = os.path.join(base_path, "../corpora/EXIST_2025_Dataset_V0.2/")
dataset_path = os.path.join(base_path, "Lab2-S1/EXIST_2025_Dataset_V0.2/")

file_train = os.path.join(dataset_path, "EXIST2025_training.json")
file_dev = os.path.join(dataset_path, "EXIST2025_dev.json")

reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)

EnTrainTask1, EnDevTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2")

SpTrainTask1, SpDevTask1 = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2 = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2")


In [ ]:
print(EnTrainTask1)

(1       200002
2       200003
5       200006
6       200007
7       200008
         ...  
3255    203256
3256    203257
3257    203258
3258    203259
3259    203260
Name: id, Length: 2870, dtype: object, 1       Writing a uni essay in my local pub with a cof...
2       @UniversalORL it is 2021 not 1921. I dont appr...
5       According to a customer I have plenty of time ...
6       So only 'blokes' drink beer? Sorry, but if you...
7       New to the shelves this week - looking forward...
                              ...                        
3255    idk why y’all bitches think having half your a...
3256    This has been a part of an experiment with @Wo...
3257    "Take me already" "Not yet. You gotta be ready...
3258    @clintneedcoffee why do you look like a whore?...
3259    ik when mandy says “you look like a whore” i l...
Name: text, Length: 2870, dtype: object, 1       YES
2       YES
5       YES
6       YES
7        NO
       ... 
3255    YES
3256    YES
3257    YES
3258    

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 25.1 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import emoji

# Regular expressions for cleaning
web_re = re.compile(r"https?:\/\/[^\s]+", re.U)  # Remove URLs
user_re = re.compile(r"(@\w+\-?(?:\w+)?)", re.U)  # Remove mentions
hashtag_re = re.compile(r"(#\w+\-?(?:\w+)?)", re.U)  # Remove hashtags
special_chars_re = re.compile(r"[^a-zA-Z0-9\s]", re.U)  # Remove special characters

def preprocess(text):
    text = pd.Series(text)  # Convert input to Pandas Series if it's not already

    text = text.str.replace(user_re, "", regex=True)  # Remove @mentions
    text = text.str.replace(web_re, "", regex=True)  # Remove URLs
    text = text.str.replace(hashtag_re, "", regex=True)  # Remove hashtags
    text = text.apply(lambda x: emoji.replace_emoji(x, replace=""))  # Remove emojis
    text = text.str.replace(special_chars_re, "", regex=True)  # Remove special characters
    text = text.str.lower().str.strip()  # Convert to lowercase and remove extra spaces

    return text  # Convert back to a list if needed

In [ ]:
EnTrainTask1 = (EnTrainTask1[0], preprocess(EnTrainTask1[1]), EnTrainTask1[2])
EnDevTask1 = (EnDevTask1[0], preprocess(EnDevTask1[1]), EnDevTask1[2])
EnTrainTask2 = (EnTrainTask2[0], preprocess(EnTrainTask2[1]), EnTrainTask2[2])
EnDevTask2 = (EnDevTask2[0], preprocess(EnDevTask2[1]), EnDevTask2[2])

## Set the seed

In [ ]:
def set_seed(seed=1234):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)
    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)

## Dataset class

In [ ]:
class SexismDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)
        }

## Metrics

In [ ]:
def compute_metrics_1(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def compute_metrics_2(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## Pipelines

In [ ]:
def sexism_classification_pipeline_task1(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=2, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc= LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_1,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    # If there is a test dataset
    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task1.csv', index=False)
        print("Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv")
        return model, submission_df
    return model, eval_results


def sexism_classification_pipeline_task2(trainInfo, devInfo, testInfo=None, model_name='bert-base-uncased', nlabels=3, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc= LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        #metric_for_best_model=args.get('metric_for_best_model',"ICM")
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_2,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    # If there is a test dataset
    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)

        })
        submission_df.to_csv('sexism_predictions_task2.csv', index=False)
        print("Prediction TASK2 completed. Results saved to sexism_predictions_task2.csv")
        return model, submission_df
    return model, eval_results


# LoRA

## LoRA pipeline subtask1

In [ ]:
######################################CHANGE###############################################
from peft import LoraConfig, get_peft_model, TaskType
###########################################################################################
def sexism_classification_pipeline_task1_LoRA(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=2, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc = LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    ######################################CHANGE###############################################
    # Configure LoRA
    lora_config = LoraConfig(
    task_type= args.get("task_type", TaskType.SEQ_CLS),
    target_modules= args.get("target_modules", ["query", "value"]),
    r= args.get("rank", 64),  # Rank of LoRA adaptation
    lora_alpha=args.get("lora_alpha", 32),  # Scaling factor
    lora_dropout=args.get("lora_dropout", 0.1),
    bias=args.get("bias", "none")
)
    ###########################################################################################

    ######################################CHANGE###############################################
    # Prepare LoRA model
    peft_model = get_peft_model(model, lora_config)

    ###########################################################################################
    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results_task1_LoRA0'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        ######################################CHANGE###############################################
        # Prepare LoRA model
        model=peft_model,
        ###########################################################################################
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_1,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    ######################################CHANGE###############################################
    #Saving the new weigths for the LoRA model
    trainer.save_model('./final_best_model_LoRA')
    # Notice that, in this case only the LoRA matrices are saved.
    # The weigths for the classification head are not saved.
    ###########################################################################################

    ######################################CHANGE###############################################
    #Mixing the LoRA matrices with the weigths of the base model used
    mixModel=peft_model.merge_and_unload()
    mixModel.save_pretrained("./final_best_model_mixpeft")
    # IN this case the full model is saved.
    ###########################################################################################

    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task1.csv', index=False)
        print("Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv")
        return model, submission_df
    return model, eval_results

## LoRA pipeline subtask2

In [ ]:
######################################CHANGE###############################################
from peft import LoraConfig, get_peft_model, TaskType
###########################################################################################
def sexism_classification_pipeline_task2_LoRA(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=3, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc = LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )
    ######################################CHANGE###############################################
    # Configure LoRA
    lora_config = LoraConfig(
    task_type= args.get("task_type", TaskType.SEQ_CLS),
    target_modules= args.get("target_modules", ["query", "value"]),
    r= args.get("rank", 64),  # Rank of LoRA adaptation
    lora_alpha=args.get("lora_alpha", 32),  # Scaling factor
    lora_dropout=args.get("lora_dropout", 0.1),
    bias=args.get("bias", "none"),
)
    ###########################################################################################

    ######################################CHANGE###############################################
    # Prepare LoRA model
    peft_model = get_peft_model(model, lora_config)

    ###########################################################################################

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results_task2_LoRA0'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        ######################################CHANGE###############################################
        # Prepare LoRA model
        model=peft_model,
        ###########################################################################################
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_2,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    ######################################CHANGE###############################################
    #Saving the new weigths for the LoRA model
    trainer.save_model('./final_best_model_LoRA')
    # Notice that, in this case only the LoRA matrices are saved.
    # The weigths for the classification head are not saved.
    ###########################################################################################

    ######################################CHANGE###############################################
    #Mixing the LoRA matrices with the weigths of the base model used
    mixModel=peft_model.merge_and_unload()
    mixModel.save_pretrained("./final_best_model_mixpeft")
    # IN this case the full model is saved.
    ###########################################################################################

    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task2.csv', index=False)
        print("Prediction for TASK 2 completed. Results saved to sexism_predictions_task1.csv")
        return model, submission_df
    return model, eval_results

# Experimental work

## English, subtask1

In [ ]:
# Usage Example
set_seed()

# COMPLETE
modelname = "roberta-base"
params = { "task_type": "SEQ_CLS", "target_modules": ["query", "value"], "rank": 64, "bias": "none" }

model, eval_results = sexism_classification_pipeline_task1(EnTrainTask1, EnDevTask1, None, modelname, 2, "single_label_classification", **params )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.431000,0.399621,0.810811,0.792079,0.761905,0.824742
2,0.392400,0.453838,0.817568,0.755287,0.912409,0.644330
3,0.401800,0.398008,0.851351,0.833333,0.816832,0.850515
4,0.138800,0.511700,0.849099,0.817439,0.867052,0.773196
5,0.224900,0.674059,0.846847,0.825641,0.821429,0.829897


Validation Results: {'eval_loss': 0.3980075418949127, 'eval_accuracy': 0.8513513513513513, 'eval_f1': 0.8333333333333334, 'eval_precision': 0.8168316831683168, 'eval_recall': 0.8505154639175257, 'eval_runtime': 2.853, 'eval_samples_per_second': 155.623, 'eval_steps_per_second': 2.454, 'epoch': 5.0}


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
# Here it is possible to define some hyperparameters used during the training and validation
# phases
set_seed()
# modelname
modelname = "cardiffnlp/twitter-roberta-base-2022-154m"
# modelname = "roberta-base"
# training params
params = {
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 16,
    "num_train_epochs": 10
}
# pipeline for the subtask1
m1, res_en1 = sexism_classification_pipeline_task1(EnTrainTask1, EnDevTask1, None, modelname,
2, "single_label_classification", **params )
# pipeline for the subtask2
# m2, res_en2 = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, None, modelname,
# 3, "single_label_classification", **params )

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.676200,0.670852,0.563063,0.000000,0.000000,0.000000
2,0.460900,0.370554,0.840090,0.822943,0.797101,0.850515
3,0.297800,0.370688,0.844595,0.824427,0.814070,0.835052
4,0.278200,0.374520,0.831081,0.792244,0.856287,0.737113
5,0.210200,0.473017,0.846847,0.813187,0.870588,0.762887


Validation Results: {'eval_loss': 0.3706875443458557, 'eval_accuracy': 0.8445945945945946, 'eval_f1': 0.8244274809160306, 'eval_precision': 0.8140703517587939, 'eval_recall': 0.8350515463917526, 'eval_runtime': 2.7746, 'eval_samples_per_second': 160.02, 'eval_steps_per_second': 2.523, 'epoch': 5.0}


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
# print(EnTrainTask1_cleaned[0][0])
# print(EnTrainTask1_cleaned[1][0])
# print(EnTrainTask1_cleaned[2][0])

writing a uni essay in my local pub with a coffee random old man keeps asking me drunk questions when im trying to concentrate amp ends with good luck but youll just end up getting married and not use it anyway  is alive and well
it is 2021 not 1921 i dont appreciate that on two rides by myself your team member looked behind me and asked the man behind how many in my party not impressed
according to a customer i have plenty of time to go spent the stirling coins he wants to pay me with in derry just like any other woman im sure of it  in retail


In [ ]:
# Usage Example
set_seed()

# COMPLETE
# modelname = "roberta-base"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.121500,1.063578,0.520548,0.327546,0.296509,0.366013
2,1.008200,0.985731,0.582192,0.245310,0.194064,0.333333


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## English, subtask2

In [ ]:

# Usage Example
set_seed()

# COMPLETE
params = {
    "learning_rate": 1e-5,
    "per_device_train_batch_size": 16,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "lr_scheduler_type": "cosine",
    "logging_steps": 20,
    "rank": 8,
    "num_train_epochs": 10
              }

model2, eval_results2 = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, None, 'bert-base-uncased', 3, "single_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.114900,1.063373,0.513699,0.318422,0.288337,0.355912
2,1.003600,0.987601,0.582192,0.245310,0.194064,0.333333
3,0.907700,0.933747,0.582192,0.245310,0.194064,0.333333
4,0.894300,0.870686,0.595890,0.300171,0.366667,0.359715
5,0.738400,0.832550,0.609589,0.375705,0.348659,0.416993
6,0.704200,0.814237,0.650685,0.439047,0.394928,0.496138
7,0.612500,0.812904,0.664384,0.456626,0.515152,0.487246
8,0.453800,0.851198,0.650685,0.542397,0.557420,0.535286
9,0.373700,0.907659,0.650685,0.517334,0.532754,0.517515
10,0.234700,1.040047,0.616438,0.525453,0.523692,0.528037


Validation Results: {'eval_loss': 0.8511983752250671, 'eval_accuracy': 0.6506849315068494, 'eval_f1': 0.542396677081098, 'eval_precision': 0.5574199507389163, 'eval_recall': 0.5352856294032765, 'eval_runtime': 1.1818, 'eval_samples_per_second': 123.538, 'eval_steps_per_second': 2.538, 'epoch': 10.0}


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
# Usage Example
set_seed()

params = {
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 64,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 3,
    "lr_scheduler_type": "polynomial",
    "logging_steps": 20,
    "rank": 4,
    "num_train_epochs": 5
              }

model3, eval_results3 = sexism_classification_pipeline_task2(EnTrainTask2, EnDevTask2, None, 'bert-base-uncased', 3, "single_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.114769,0.328767,0.242136,0.368733,0.379798
2,1.131500,1.077275,0.479452,0.337758,0.315917,0.398099
3,1.078500,1.039685,0.575342,0.290862,0.308191,0.347950
4,1.078500,1.006681,0.582192,0.245310,0.194064,0.333333
5,1.022700,0.974145,0.582192,0.245310,0.194064,0.333333


Validation Results: {'eval_loss': 1.0772746801376343, 'eval_accuracy': 0.4794520547945205, 'eval_f1': 0.3377582917224861, 'eval_precision': 0.3159166031019578, 'eval_recall': 0.39809863339275103, 'eval_runtime': 1.0925, 'eval_samples_per_second': 133.634, 'eval_steps_per_second': 2.746, 'epoch': 5.0}


## English with LoRA, subtask1

In [ ]:

# Usage Example
set_seed()

# COMPLETE

modelname = "bert-base-uncased"
params = {
    "num_train_epochs": 8,
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "logging_steps": 20,
    "rank": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model4, eval_results4 = sexism_classification_pipeline_task1_LoRA(EnTrainTask1, EnDevTask1, None, modelname, 2, "single_label_classification", **params )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.709600,0.695146,0.513514,0.379310,0.428571,0.340206
2,0.697300,0.688433,0.572072,0.151786,0.566667,0.087629
3,0.690100,0.689135,0.567568,0.030303,0.750000,0.015464
4,0.673200,0.687341,0.567568,0.020408,1.000000,0.010309
5,0.675500,0.684548,0.565315,0.010256,1.000000,0.005155


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1107: UserWarning: Unable to fetch remote file due to the following error 500 Server Error: Internal Server Error for url: https://huggingface.co/bert-base-uncased/resolve/main/config.json (Request ID: Root=1-67fe29b2-6adf05f65b6897233c4b3e30;fe78e008-583d-437a-a285-71445cca0055)

Internal Error - We're working hard to fix this as soon as possible! - silently ignoring the lookup for the file config.json in bert-base-uncased.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in bert-base-uncased - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1107: UserWarning: Unable to fetch remote file due to the following error 500 Server Error: Internal Server Error for url: https://huggingface.co/bert-base-uncased/resolve/main/config.json (Request ID: Root=1-67fe29de-7ee56c9f60b

Validation Results: {'eval_loss': 0.6951464414596558, 'eval_accuracy': 0.5135135135135135, 'eval_f1': 0.3793103448275862, 'eval_precision': 0.42857142857142855, 'eval_recall': 0.3402061855670103, 'eval_runtime': 3.2863, 'eval_samples_per_second': 135.105, 'eval_steps_per_second': 2.13, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1107: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 86afac8d-3223-4657-9be0-ac838040b20c)') - silently ignoring the lookup for the file config.json in bert-base-uncased.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in bert-base-uncased - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
# Run it

# Usage Example
set_seed()

# COMPLETE

modelname = "roberta-base"
params = {
    "num_train_epochs": 10,
    "learning_rate": 4e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 4,
    "logging_steps": 20,
    "rank": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model5, eval_results5 = sexism_classification_pipeline_task1_LoRA(EnTrainTask1, EnDevTask1, None, modelname, 2, "single_label_classification", **params )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
"""
from peft import PeftModel # importing the PeftModel class
# The model can be loadded in a simple way.
model = AutoModelForSequenceClassification.from_pretrained("./final_best_model_mixpeft")
"""

'\nfrom peft import PeftModel # importing the PeftModel class\n# The model can be loadded in a simple way.\nmodel = AutoModelForSequenceClassification.from_pretrained("./final_best_model_mixpeft")\n'

## English with LoRA, subtask2

In [ ]:
# Run it

# Usage Example
set_seed()

# COMPLETE

modelname = "roberta-base"
params = {
    "num_train_epochs": 10,
    "learning_rate": 4e-5,
    "per_device_train_batch_size": 16,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 4,
    "lora_alpha": 16,
    "lora_dropout": 0.1
}
model6, eval_results6= sexism_classification_pipeline_task2_LoRA(EnTrainTask2, EnDevTask2, None, modelname, 3, "single_label_classification", **params )

In [ ]:
# Run it

# Usage Example
set_seed()

# COMPLETE

modelname = "roberta-base"
params = {
    "num_train_epochs": 10,
    "learning_rate": 1e-5,
    "per_device_train_batch_size": 32,
    "weight_decay": 0.05,
    "eval_strategy": "epoch",
    "early_stopping_patience": 2,
    "logging_steps": 20,
    "rank": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.05
}
model7, eval_results7 = sexism_classification_pipeline_task2_LoRA(EnTrainTask2, EnDevTask2, None, modelname, 3, "single_label_classification", **params )

### Not necessary

# Do it in Spanish

In [ ]:





# COMPLETE, use as many cells as you need







# Show results

In [ ]:


# COMPLETE, Show the information you find useful



English
Fine-tuning:
	subtask1:0.8146 (86.52 per epoch)
	subtask2:0.6007 (29.31 per epoch)
LoRA (Low Rank Adaptation):
	subtask1:0.8363 (59.53 per epoch)
	subtask2:0.5729 (18.18 per epoch)


In [ ]:
# COMPLETE

English
Fine-tuning:
	subtask1:0.8365 (85.25 per epoch)
	subtask2:0.6611 (30.79 per epoch)
LoRA (Low Rank Adaptation):
	subtask1:0.8384 (60.57 per epoch)
	subtask2:0.5894 (18.4 per epoch)


In [ ]:
# COMPLETE

Spanish
Fine-tuning:
	subtask1:0.7774 (96.84 per epoch)
	subtask2:0.6394 (37.88 per epoch)
LoRA (Low Rank Adaptation):
	subtask1:0.8811 (67.46 per epoch)
	subtask2:0.663 (26.01 per epoch)


In [ ]:
# COMPLETE

Spanish
Fine-tuning:
	subtask1:0.8398 (92.42 per epoch)
	subtask2:0.5886 (36.53 per epoch)
LoRA (Low Rank Adaptation):
	subtask1:0.8305 (66.66 per epoch)
	subtask2:0.6316 (25.87 per epoch)


In [ ]:

# Add all you want/need/consider
